In [44]:
import requests
import numpy as np
from collections import deque
from bs4 import BeautifulSoup

In [123]:
def getneighbors(username):
    url = f"https://www.last.fm/user/{username}/neighbours"
    resp = requests.get(url)    
    if resp.status_code==200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        us = soup.find("section", {"class": "neighbours-items-section"})
        us = us.findAll("a", {"class": "user-list-link link-block-target"})
        return np.asarray([u.text for u in us])
    else:
        return np.array([])

In [122]:
getneighbors("aidan_bbq")

array(['laurenmcroy', 'millierockwho', 'paulaardissone', 'Lrov1023',
       'audiostomper', 'Jojolovesmomos', 'salvadorable3', 'ellethmpsn',
       'zoyaleyla', 'tharsell', 'lemonsforever5', 'nalshalash', 'Yegtr',
       'gesutton7', 'Amnowls13', 'annalee_k', 'archiecarlow',
       'Drewmuench', 'mellow22_', 'ddziam', 'Eamegill', 'ragepiss',
       'mia_palas', 'Aidanp123456', 'Aidanp1234567', 'terabyte99',
       'Nijamo', 'gary_lannon', 'aliamirkov', 'elizab5th', 'bebibube',
       'jacoboosh', 'Cole199910', 'o_lukas', 'TiesTulp', 'Jqckvinceette',
       'bidsalvaterra_', 'Hasuso1', 'theowalker', 'Tyjy0', 'uvbrux123',
       'w1ll715', 'dnmphy', 'markimeyer95', 'J_Coggins', 'johnstatsick',
       'Kickyouass99', 'greenishpurple', 'jamn200', 'NanoTechhhhh'],
      dtype='<U14')

In [130]:
def scrapeusers(userheads = ["aidan_bbq"], maxdepth = 1, maxusers = 100):
    searchstack = deque(userheads)
    useredges = None # to make graph of users
    userset = set(userheads)
    explored = np.array([]) 
    
    depth = 0
    numleft = len(userheads) # to calculate depth, number of edges left to explore
    with open("./output/neighbors.txt", 'a') as f:
        while depth <= maxdepth and len(searchstack) > 0 and len(userset) < maxusers:
            curruser = searchstack.popleft()
            explored = np.append(explored, curruser)

            neighs = getneighbors(curruser)
            userset = userset.union(neighs)

            edges = np.asarray([np.repeat(curruser, len(neighs)), neighs]).T
            [f.write(f"{e[0]}	{e[1]}\n") for e in edges]
            if useredges is None:
                useredges = edges
            else:
                useredges = np.concatenate((useredges, edges))

            searchstack.extend(neighs[~np.isin(neighs, explored)]) # get rid of already visited neighs
            numleft -= 1
            if numleft == 0: # if we've explored entire level, increase depth and add next level
                numleft = len(searchstack)
                depth += 1
    return userset, useredges
scrapeusers(maxusers = 300)

({'Aidanp123456',
  'Aidanp1234567',
  'Amnowls13',
  'AndreDenji',
  'Arlkara',
  'BBaloni',
  'Bestrada_',
  'Busin64',
  'C_chamomile',
  'Chavasse1928',
  'Cole199910',
  'Collegewifi',
  'CollinH678',
  'DUDEZ3RO',
  'DUM0RE',
  'Dadpoohbear',
  'DannysPhantom',
  'DarkJkl2009',
  'Digbuster',
  'Drewmuench',
  'Eamegill',
  'Eastland_alex',
  'Edpud',
  'Eimearcarey3',
  'Elenaenoelle',
  'Emma_rh',
  'Ethan-edwards',
  'F_Hosier',
  'FaffoutYDG',
  'Fazooka',
  'Fivecentboto',
  'Focusillusion',
  'Fosterthefish',
  'G1oomi',
  'Gazasmusic',
  'GirthyAsian',
  'Griffybiffy',
  'HAMBONE32',
  'Hasuso1',
  'Ishaboicollin',
  'J_Coggins',
  'JasonC2004',
  'Jhunt1994',
  'Jojolovesmomos',
  'Jqckvinceette',
  'KenRosenbergX',
  'Kickyouass99',
  'Lrov1023',
  'Luissss5S',
  'Mtinetta',
  'Namigva',
  'NanoTechhhhh',
  'NewAgeJeff',
  'Nijamo',
  'Nomkey',
  'Oliviabucky',
  'OnigiriEater',
  'Panic',
  'PatLavey',
  'Paulineg13',
  'Pedgyyy',
  'Phebe123',
  'PokeyTheLilPup',
  'Ro